In [36]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
testdata = sm.datasets.get_rdataset("dietox", "geepack").data
testdata

,Pig,Evit,Cu,Litter,Start,Weight,Feed,Time
0,4601,Evit000,Cu000,1,26.5,26.50000,NaN,1
1,4601,Evit000,Cu000,1,26.5,27.59999,5.200005,2
2,4601,Evit000,Cu000,1,26.5,36.50000,17.600000,3
3,4601,Evit000,Cu000,1,26.5,40.29999,28.500000,4
4,4601,Evit000,Cu000,1,26.5,49.09998,45.200001,5
...,...,...,...,...,...,...,...,...
856,8442,Evit000,Cu175,24,25.7,73.19995,83.800003,8
857,8442,Evit000,Cu175,24,25.7,81.69995,99.800003,9
858,8442,Evit000,Cu175,24,25.7,90.29999,115.200001,10
859,8442,Evit000,Cu175,24,25.7,96.00000,133.200001,11


In [12]:
md = smf.mixedlm("Weight ~ Time", testdata, groups=testdata["Pig"])
mdf = md.fit()
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: Weight    
No. Observations: 861     Method:             REML      
No. Groups:       72      Scale:              11.3669   
Min. group size:  11      Log-Likelihood:     -2404.7753
Max. group size:  12      Converged:          Yes       
Mean group size:  12.0                                  
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    15.724    0.788  19.952 0.000 14.179 17.268
Time          6.943    0.033 207.939 0.000  6.877  7.008
Group Var    40.394    2.149                            
========================================================

"""

## Lipid analysis

In [37]:
fpath = '../../resources/lipid-data-species-normalized-pretty.csv'
norm_lipid_species = pd.read_csv(fpath, header=[0, 1], index_col=[0, 1])
fpath = '../../resources/lipid-data-species-pretty.csv'
lipid_species = pd.read_csv(fpath, header=[0, 1], index_col=[0, 1])

In [51]:
s = lipid_species.stack(level=1).stack().rename_axis(['Group', 'Species', 'Region', 'Subject'])
endog = s.to_frame('Level')
exog = s.index.to_frame()
exog['Dx'] = [re.sub('^(Control|ChAc).*$', '\\1', s) for s in exog.Subject]
data = pd.concat([endog, exog], axis=1)
data

Level Group     Species   Region  \
Group Species    Region  Subject                                          
FC    FC         CN      ChAc 1     74.76526    FC          FC       CN   
                         ChAc 2     62.75982    FC          FC       CN   
                         ChAc 3     85.05140    FC          FC       CN   
                         ChAc 4     85.80184    FC          FC       CN   
                         Control 1  86.01249    FC          FC       CN   
...                                      ...   ...         ...      ...   
NSer  NSer 22:6  Putamen Control 2   0.00024  NSer  NSer 22:6   Putamen   
                         Control 3   0.00074  NSer  NSer 22:6   Putamen   
                         Control 4   0.00008  NSer  NSer 22:6   Putamen   
                         Control 5   0.00034  NSer  NSer 22:6   Putamen   
                         Control 6   0.00042  NSer  NSer 22:6   Putamen   

                                      Subject       Dx  
Group Species    Region  Subject                        
FC    FC         CN      ChAc 1        ChAc 1     ChAc  
                         ChAc 2        ChAc 2     ChAc  
                         ChAc 3        ChAc 3     ChAc  
                         ChAc 4        ChAc 4     ChAc  
                         Control 1  Control 1  Control  
...                                       ...      ...  
NSer  NSer 22:6  Putamen Control 2  Control 2  Control  
                         Control 3  Control 3  Control  
                         Control 4  Control 4  Control  
                         Control 5  Control 5  Control  
                         Control 6  Control 6  Control  

[17790 rows x 6 columns]

In [5]:
%connect_info

{
  "shell_port": 58738,
  "iopub_port": 58739,
  "stdin_port": 58740,
  "control_port": 58742,
  "hb_port": 58741,
  "ip": "127.0.0.1",
  "key": "cb655d5b-0723ead18e1739c75c260065",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-92dc17a4-3ede-42d4-8ab8-607ddeecd669.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
